En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I.

In [29]:
import requests
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

In [30]:
df_completo=pd.read_csv("./datos_union_clima_ataques.csv",index_col=0)
df_completo.reset_index(inplace=True)
df_completo.head()


,level_0,case_number,year,type,country,area,location,activity,name,age,...,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
1,1,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
2,2,2018.06.09,2018.0,Invalid,USA,Hawaii,"Habush, Oahu",Surfing,John Denges,48,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
3,3,2018.05.27,2018.0,Unprovoked,USA,Florida,"Lighhouse Point Park, Ponce Inlet, Volusia County",Fishing,male,52,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875
4,4,2018.05.26.b,2018.0,Unprovoked,USA,Florida,"Cocoa Beach, Brevard County",Walking,Cody High,15,...,-9999.0,-9999.0,11.421875,9.6875,4.25,1013.703125,1.234375,0.0,192.109375,2.921875


In [31]:
# pais= "USA","Australia","South Africa","New Zealand","Papua New Guinea"
# latitud_longitud= [39.7837304,-100.445882], [-24.7761086,134.755],[-28.8166236,24.991639], [-41.5000831,172.8344077], [-5.6816069,144.2489081]

# diccionario = dict(zip(pais, latitud_longitud))

In [32]:
lat = 39.7837304
lon = -100.445882
producto = "meteo"
pais = "USA"

In [33]:
# for key, value in diccionario.items():
#     lon= value[1]
#     lat= value[0]
#     producto= "meteo"

In [34]:
class Extraccion: 

    def __init__(self, lat, lon, pais):
        self.lat = lat
        self.lon = lon
        self.pais = pais
        self.producto = producto


    def llamar_API(self,producto):
        self.producto = producto

        url = f'http://www.7timer.info/bin/api.pl?lon=-{self.lon}&lat={self.lat}&product={producto}&output=json' 

        response = requests.get(url=url)
        response.status_code
        response.reason

        #Creamos el dataFrame que viene de la API
        df_API = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
        return df_API


    def juntar_df(self, df_completo, df_API): 
    
        df_API["pais"]=self.pais
        df_API["latitud"]=self.lat
        df_API["longitud"]=self.lon

        df_completo = pd.concat([df_completo,df_API], axis = 0)
    
        # # guardamos los datos
        #df_completo.to_pickle('datos/datos_actualizados.pkl')
     

        return df_completo
    

    def insertar_fecha(self, df_completo): 

        hoy = datetime.now()
        hoy = datetime.strftime(hoy, '%Y-%m-%d') # va a estar en formato YYYY-MM-DD

        # creamos la nueva columna
        df_completo["fecha"] = hoy #Creamos una columna dentro del DF
        


    def chequear_datos(self, df): 
    
        print("Las columnas son:", "\n")
        print(list(df_completo.columns))
        print("-----------------------------------------")

        print("Los tipos de datos que tenemos son:", "\n")
        print(df_completo.dtypes)
        print("-----------------------------------------")

        print("El porcentaje de nulos:", "\n")
        print((df_completo.isnull().sum() / df_completo.shape[0]) *  100)

        print("El porcentaje de duplicados:", "\n")
        print(df_completo.duplicated().sum())




    # def limpiar_dataframe(self, df, lista_columnas): 

    #     #convertimos la fecha a datetime
    #     df_completo["fecha"] = pd.to_datetime(df_completo["fecha"])

    #     # reemplazamos los nulos de las columnas por la media
    #      # lista de columnas en las que queremos reemplazar los nulos
    #     df_completo[lista_columnas]=df_completo[lista_columnas].fillna(df.mean())

    #     # quitar columnas repetidas:
        
    #     df.drop(["ciudad_y"], axis = 1, inplace = True)
        
    #     # renombrar columnas
        
    #     df.rename(columns = {"ciudad_x": "ciudad"}, inplace = True)

    #     # quitar % 
    #     df["rh2m"] = df["rh2m"].replace(r"%", "", regex = True)

    #     # guardamos los datos una vez limpios
    #     df.to_pickle('datos/datos_Madrid.pkl')
    #     df.to_csv('datos/datos_Madrid.csv')


In [35]:
api = Extraccion(lat, lon, pais)

In [36]:
df_API = api.llamar_API("meteo")
df_API.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 7}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 75, 'speed': ...",22,2,6,1020,none,0,0,85,3
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 6}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 110, 'speed':...",22,2,5,1018,none,0,0,100,3


In [37]:
df_completo= api.juntar_df(df_completo,df_API) 
df_completo.head(1)

,level_0,case_number,year,type,country,area,location,activity,name,age,...,msl_pressure,prec_amount,snow_depth,wind10m.direction,wind10m.speed,rh_profile,wind_profile,prec_type,latitud,longitud
0,0.0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,...,1013.703125,1.234375,0.0,192.109375,2.921875,NaN,NaN,NaN,NaN,NaN


In [38]:
api.insertar_fecha(df_completo)
df_completo.head(2)

,level_0,case_number,year,type,country,area,location,activity,name,age,...,prec_amount,snow_depth,wind10m.direction,wind10m.speed,rh_profile,wind_profile,prec_type,latitud,longitud,fecha
0,0.0,2018.06.25,2018.0,Boating,USA,California,"Oceanside, San Diego County",Paddling,Julie Wolfe,57,...,1.234375,0.0,192.109375,2.921875,NaN,NaN,NaN,NaN,NaN,2022-05-18
1,1.0,2018.06.18,2018.0,Unprovoked,USA,Georgia,"St. Simon Island, Glynn County",Standing,AdysonÂ McNeely,11,...,1.234375,0.0,192.109375,2.921875,NaN,NaN,NaN,NaN,NaN,2022-05-18


In [39]:
api.chequear_datos(df_completo) #Voy por aqui

Las columnas son: 

['level_0', 'case_number', 'year', 'type', 'country', 'area', 'location', 'activity', 'name', 'age', 'time', 'species_', 'injury', 'fecha_limpia', 'fatal', 'sex', 'pais', 'rh_950mb', 'rh_900mb', 'rh_850mb', 'rh_800mb', 'rh_750mb', 'rh_700mb', 'rh_650mb', 'rh_600mb', 'rh_550mb', 'rh_500mb', 'rh_450mb', 'rh_400mb', 'rh_350mb', 'rh_300mb', 'rh_250mb', 'rh_200mb', 'wind_950mb', 'wind_900mb', 'wind_850mb', 'wind_800mb', 'wind_750mb', 'wind_700mb', 'wind_650mb', 'wind_600mb', 'wind_550mb', 'wind_500mb', 'wind_450mb', 'wind_400mb', 'wind_350mb', 'wind_300mb', 'wind_250mb', 'wind_200mb', 'index', 'timepoint', 'cloudcover', 'highcloud', 'midcloud', 'lowcloud', 'temp2m', 'lifted_index', 'rh2m', 'msl_pressure', 'prec_amount', 'snow_depth', 'wind10m.direction', 'wind10m.speed', 'rh_profile', 'wind_profile', 'prec_type', 'latitud', 'longitud', 'fecha']
-----------------------------------------
Los tipos de datos que tenemos son: 

level_0         float64
case_number      object


TypeError: unhashable type: 'list'